In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy.interpolate import make_interp_spline
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap
from scipy.stats import gaussian_kde

In [ ]:
df = pd.read_csv('../result/output_scenario.csv')
df['pop'] = np.where(df['pop'] > 500, 500, df['pop'])
df['total_cost_0_ave'] = df['total_cost_0'] / df['pop']
df['total_cost_2020_ave'] = df['total_cost_2020'] / df['pop']
df['total_cost_2050_ave'] = df['total_cost_2050'] / df['pop']

# 计算两种差异指标
df['diff_absolute'] = df['total_cost_2020_ave'] - df['total_cost_0_ave']  # 绝对差值
df['diff_relative1'] = (df['total_cost_2020_ave'] - df['total_cost_0_ave']) / df['total_cost_0_ave']  # 相对变化比例
df['diff_relative2'] = (df['total_cost_2050_ave'] - df['total_cost_2020_ave']) / df['total_cost_2020_ave']  # 相对变化比例

df.head()

In [ ]:
print(df['diff_absolute'].describe())
print(df['diff_relative1'].describe())
print(df['diff_relative2'].describe())

In [ ]:
def visualize_diff(data, vmax, vmin, column, title_suffix="", colorbar_label=""):
    plt.rcParams['font.family'] = 'Times New Roman'
    # 导入必要的模块
    from matplotlib.offsetbox import OffsetImage, AnnotationBbox
    import numpy as np
    from scipy.ndimage import gaussian_filter
    
    # 创建Point几何图形
    data['geometry'] = data.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
    geo_df = gpd.GeoDataFrame(data, geometry='geometry')
    geo_df.set_crs(epsg=4326, inplace=True)
    
    # 创建带有cartopy投影的图形
    fig = plt.figure(figsize=(14, 6), dpi=500)
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson())

    # 设置背景和地图特征
    ax.set_facecolor('#FFFFFF')  
    ax.add_feature(cfeature.LAND, color="#CECECE", alpha=0.4)
    ax.add_feature(cfeature.OCEAN, color="#FFFFFF", alpha=0.5)
    ax.add_feature(cfeature.COASTLINE, linewidth=0.7)
    ax.add_feature(cfeature.BORDERS, linestyle=':', linewidth=0.5, alpha=0.3)
    
    # 使用更简单的方法创建密度图
    resolution = 1.0  # 1度分辨率
    lons = np.arange(-180, 181, resolution)
    lats = np.arange(-90, 91, resolution)
    
    # 创建网格点
    lon_grid, lat_grid = np.meshgrid(lons, lats)
    density_grid = np.zeros_like(lon_grid)
    
    # 提取所有点的经纬度和值
    all_lons = geo_df.geometry.x.values
    all_lats = geo_df.geometry.y.values
    all_values = np.abs(geo_df[column].values)  # 使用绝对值用于密度计算
    
    # 使用更适合地理数据的影响半径
    bandwidth = 2.0
    
    # 计算每个网格点的密度值
    for i, row in enumerate(all_lons):
        lon, lat = all_lons[i], all_lats[i]
        value = all_values[i]
        
        # 计算此数据点对网格的影响
        for x in range(len(lats)):
            for y in range(len(lons)):
                grid_lon = lons[y]
                grid_lat = lats[x]
                
                # 计算距离
                dist = np.sqrt((grid_lon - lon)**2 + (grid_lat - lat)**2)
                
                # 只在一定半径内影响网格点
                if dist <= 3 * bandwidth:
                    # 使用高斯核计算权重
                    weight = np.exp(-0.5 * (dist / bandwidth)**2) * value
                    density_grid[x, y] += weight
    
    # 对密度图应用轻微的高斯平滑
    density_grid = gaussian_filter(density_grid, sigma=1.0)
    
    # 设置阈值，只在密度足够高的区域绘制斜线
    threshold = np.max(density_grid) * 0.15
    mask = density_grid < threshold
    
    # 设置上限值，避免极端值影响色彩分布
    # 使用百分位数而不是最大值来设置上限，避免离群值的影响
    upper_limit = np.percentile(density_grid[~mask], 99)  # 使用99%分位数作为上限
    
    # 裁剪过高的值
    density_grid_capped = np.clip(density_grid, 0, upper_limit)
    
    # 创建掩码数组
    density_grid_masked = np.ma.array(density_grid_capped, mask=mask)
    
    # 热力图颜色 - 使用更多颜色变化
    heatmap_colors = ['#012f4830', '#669aba40', '#fbf0d950', '#be142060', '#7a010170']
    heatmap_cmap = mpl.colors.LinearSegmentedColormap.from_list('heatmap_palette', heatmap_colors)
    
    # 创建自定义规范化对象，确保色彩分布均匀
    heatmap_norm = mpl.colors.Normalize(vmin=threshold, vmax=upper_limit)
    
    # 绘制密度图 - 只在有数据的区域绘制斜线
    contour = ax.contourf(
        lon_grid, lat_grid, density_grid_masked,
        transform=ccrs.PlateCarree(),
        cmap=heatmap_cmap,
        norm=heatmap_norm,  # 使用自定义规范化
        levels=8,
        alpha=0.7,
        zorder=2,
        extend='neither'
    )
    
    # 设置颜色映射 - 修改为蓝-白-红，适合显示变化差异
    diff_colors = ['#012f48', '#669aba', '#ffffff', '#be1420', '#7a0101']  # 蓝-白-红
    custom_cmap = mpl.colors.LinearSegmentedColormap.from_list('diff_palette', diff_colors)
    norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)  # 使用对称范围
    
    # 修改点大小范围和计算方式
    min_size = 25  # 最小点大小
    max_size = 200 # 限制最大点大小
    red_multiplier = 1.5  # 红色点的放大倍数
    
    # 对数据按绝对值排序，保证小值在下层，大值在上层
    geo_df = geo_df.sort_values(by=column, key=abs)
    
    # 创建一个假散点用于颜色条
    fake_scatter = ax.scatter(
        [-1000], [-1000],  # 不可见位置
        c=[0],
        cmap=custom_cmap,
        vmin=vmin,
        vmax=vmax,
        s=1
    )
    
    # 为每个点创建并添加标记
    for idx, row in geo_df.iterrows():
        # 获取经纬度和值
        lon, lat = row.geometry.x, row.geometry.y
        value = row[column]
        
        # 计算点大小基于绝对值
        abs_value = abs(value)
        base_size = min_size + ((abs_value / vmax) * (max_size - min_size))
        
        # 如果是红色点（正值），增大点的大小
        if value > 0:
            size = min(base_size * red_multiplier, max_size * 1.2)  # 红色点更大，但不超过限制
        else:
            size = base_size
            
        # 确保不超过绝对最大值
        size = min(size, 200)  # 绝对最大点大小限制
        
        # 计算颜色基于实际值（包括正负）
        color = custom_cmap(norm(value))
        
        # 创建临时图形来生成点图像
        temp_fig = plt.figure(figsize=(1, 1), frameon=False, dpi=300)
        temp_fig.patch.set_alpha(0)  # 透明背景
        
        temp_ax = temp_fig.add_subplot(111)
        temp_ax.set_aspect('equal')
        temp_ax.patch.set_alpha(0)
        
        # 先绘制一个稍大的黑色圆作为边框
        outer_circle = plt.Circle(
            (0.5, 0.5),
            0.18,         # 稍大的半径
            color='black',  # 黑色
            alpha=1
        )
        temp_ax.add_patch(outer_circle)

        # 再绘制一个内圆作为主要颜色区域
        inner_circle = plt.Circle(
            (0.5, 0.5),
            0.15,         # 稍小的半径
            color=color,  # 数据颜色
            alpha=1
        )
        temp_ax.add_patch(inner_circle)
        
        # 设置坐标轴范围
        temp_ax.set_xlim(0, 1)
        temp_ax.set_ylim(0, 1)
        temp_ax.axis('off')  # 隐藏坐标轴
        
        # 渲染并获取图像
        temp_fig.tight_layout(pad=0)
        temp_fig.canvas.draw()
        point_img = np.array(temp_fig.canvas.renderer.buffer_rgba())
        plt.close(temp_fig)
        
        # 将地理坐标转换为投影坐标
        x, y = ax.projection.transform_point(lon, lat, src_crs=ccrs.PlateCarree())
        
        # 计算缩放因子 - 调整点大小
        zoom_factor = np.sqrt(size) / 200  # 调整缩放因子
        
        # 创建OffsetImage
        imagebox = OffsetImage(point_img, zoom=zoom_factor)
        imagebox.image.axes = ax
        
        # 创建并添加AnnotationBbox
        ab = AnnotationBbox(
            imagebox,
            (x, y),
            frameon=False,
            pad=0,
            zorder=10  # 确保点在最上层
        )
        ax.add_artist(ab)
    
    # 添加颜色条
    cbar = fig.colorbar(
        fake_scatter,
        ax=ax,
        orientation='horizontal',
        shrink=0.6,  # 控制颜色条长度
        pad=0.03,    # 调整颜色条和图形之间的间距
        aspect=50    # 控制颜色条的宽度（值越小越宽）
    )
    cbar.set_label(colorbar_label, fontsize=12)
    
    # 设置标题
    if title_suffix:
        ax.set_title(f'Change in Per Capita Investment {title_suffix}', 
                    fontsize=14, fontweight='bold', pad=20)
    
    # 设置全球边界
    ax.set_global()
     
    # 移除坐标轴刻度和边框
    ax.set_xticks([])
    ax.set_yticks([])
    for spine in ax.spines.values():
        spine.set_visible(False)
    plt.axis('off')  # 完全关闭坐标轴
    
    # 减少边距
    plt.tight_layout(pad=0)
    
    plt.show()
    
    # 打印统计信息
    print(f"\nVisualization Statistics for {title_suffix}:")
    print(f"  Total data points: {len(geo_df)}")
    print(f"  Value range: {geo_df[column].min():.6f} to {geo_df[column].max():.6f}")
    print(f"  Positive changes: {(geo_df[column] > 0).sum()}")
    print(f"  Negative changes: {(geo_df[column] < 0).sum()}")
    print(f"  No change: {(geo_df[column] == 0).sum()}")
    print(f"  Mean: {geo_df[column].mean():.6f}")
    print(f"  Median: {geo_df[column].median():.6f}")
    print(f"  Standard deviation: {geo_df[column].std():.6f}")
    
    return fig

In [ ]:
# 第一张地图：相对百分比变化（2020-0）
vmax_rel = max(abs(df['diff_relative1'].min()), abs(df['diff_relative1'].max()))
vmax_rel = min(vmax_rel, np.percentile(np.abs(df['diff_relative1']), 99))  # 使用95%分位数避免极值影响
vmin1 = -0.01

visualize_diff(df, vmax_rel, vmin1, 'diff_relative1', 
              title_suffix="(Relative Difference between No Risk-based and Current Risk-based)", 
              colorbar_label="Relative Change in Per Capita Investment ($)")


# 第二张地图：相对百分比变化（2050-2020）
vmax_rel = max(abs(df['diff_relative2'].min()), abs(df['diff_relative2'].max()))
vmax_rel = min(vmax_rel, np.percentile(np.abs(df['diff_relative2']), 99))  # 使用95%分位数避免极值影响
vmin2= -0.1

visualize_diff(df, vmax_rel, vmin2, 'diff_relative2',
              title_suffix="(Relative Difference between Current  Risk-based and Future Risk-based)", 
              colorbar_label="Relative Change in Per Capita Investment (%)")

In [ ]:
def create_longitude_profile(data, column, ax=None, title_suffix=""):
    """创建经度方向的差异分布曲线图"""
    plt.rcParams['font.family'] = 'Times New Roman'
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(14, 2),dpi=500)
    
    # 按经度分组计算平均值
    lon_bins = np.arange(-180, 181, 5)  # 5度一组
    lon_centers = (lon_bins[:-1] + lon_bins[1:]) / 2
    lon_values = []
    
    for i in range(len(lon_bins)-1):
        mask = (data['lon'] >= lon_bins[i]) & (data['lon'] < lon_bins[i+1])
        if mask.any():
            lon_values.append(data.loc[mask, column].mean())
        else:
            lon_values.append(0)  # 用0代替NaN
    
    # 平滑曲线
    if len(lon_centers) > 3:
        lon_smooth = np.linspace(min(lon_centers), max(lon_centers), 300)
        spline = make_interp_spline(lon_centers, lon_values, k=3)
        lon_values_smooth = spline(lon_smooth)
    else:
        lon_smooth = lon_centers
        lon_values_smooth = lon_values
    
    # 根据数据类型确定是否允许负值和颜色
    if column in ['diff_absolute', 'diff_relative']:
        # 差异数据可以有负值
        y_min = min(lon_values_smooth) * 1.1 if min(lon_values_smooth) < 0 else 0
        y_max = max(lon_values_smooth) * 1.1 if max(lon_values_smooth) > 0 else 0
        
        # 分别绘制正值和负值区域
        positive_mask = lon_values_smooth >= 0
        negative_mask = lon_values_smooth < 0
        
        # 绘制正值区域（红色）
        if np.any(positive_mask):
            ax.fill_between(lon_smooth, 0, lon_values_smooth, 
                          where=positive_mask, alpha=0.3, color="#be1420", 
                          interpolate=True, label='Increase')
        
        # 绘制负值区域（蓝色）
        if np.any(negative_mask):
            ax.fill_between(lon_smooth, 0, lon_values_smooth, 
                          where=negative_mask, alpha=0.3, color="#012f48", 
                          interpolate=True, label='Decrease')
        
        # 绘制曲线
        ax.plot(lon_smooth, lon_values_smooth, color='black', linewidth=1.5)
        
        # 添加零线
        ax.axhline(y=0, color='gray', linestyle='-', alpha=0.5, linewidth=1)
        
    else:
        # 正值数据（如原始投资额）
        y_min = 0
        y_max = max(lon_values_smooth) * 1.1
        
        # 绘制曲线和填充
        ax.plot(lon_smooth, lon_values_smooth, color='#7a0101', linewidth=1.5)
        ax.fill_between(lon_smooth, 0, lon_values_smooth, alpha=0.3, color="#a37070")
    
    # 设置范围
    ax.set_xlim(-180, 180)
    ax.set_ylim(y_min, y_max)
    
    # 设置标题和标签
    ax.set_title(f'Longitude Distribution {title_suffix}', fontsize=12, fontweight='bold')
    # ax.set_xlabel('Longitude (°)', fontsize=10)
    ax.set_xlabel('')  # 移除x轴标题
    ax.set_xticklabels([]) 
    ax.tick_params(axis='x', length=0)
    # 美化图表
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.grid(False)
    
    # 如果有图例，显示它
    # if column in ['diff_absolute', 'diff_relative'] and (np.any(lon_values_smooth > 0) and np.any(lon_values_smooth < 0)):
    #     ax.legend(fontsize=9)
    
    return ax

def create_latitude_profile(data, column, ax=None, title_suffix=""):
    """创建纬度方向的差异分布曲线图"""
    plt.rcParams['font.family'] = 'Times New Roman'
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(2.5, 10),dpi=500)
    
    # 按纬度分组计算平均值
    lat_bins = np.arange(-90, 91, 2)  # 2度一组
    lat_centers = (lat_bins[:-1] + lat_bins[1:]) / 2
    lat_values = []
    
    for i in range(len(lat_bins)-1):
        mask = (data['lat'] >= lat_bins[i]) & (data['lat'] < lat_bins[i+1])
        if mask.any():
            lat_values.append(data.loc[mask, column].mean())
        else:
            lat_values.append(0)  # 用0代替NaN
    
    # 平滑曲线
    if len(lat_centers) > 3:
        lat_smooth = np.linspace(min(lat_centers), max(lat_centers), 300)
        spline = make_interp_spline(lat_centers, lat_values, k=3)
        lat_values_smooth = spline(lat_smooth)
    else:
        lat_smooth = lat_centers
        lat_values_smooth = lat_values
    
    # 根据数据类型确定是否允许负值和颜色
    if column in ['diff_absolute', 'diff_relative']:
        # 差异数据可以有负值
        x_min = min(lat_values_smooth) * 1.1 if min(lat_values_smooth) < 0 else 0
        x_max = max(lat_values_smooth) * 1.1 if max(lat_values_smooth) > 0 else 0
        
        # 分别绘制正值和负值区域
        positive_mask = lat_values_smooth >= 0
        negative_mask = lat_values_smooth < 0
        
        # 绘制正值区域（红色）
        if np.any(positive_mask):
            ax.fill_betweenx(lat_smooth, 0, lat_values_smooth, 
                           where=positive_mask, alpha=0.3, color="#be1420", 
                           interpolate=True, label='Increase')
        
        # 绘制负值区域（蓝色）
        if np.any(negative_mask):
            ax.fill_betweenx(lat_smooth, 0, lat_values_smooth, 
                           where=negative_mask, alpha=0.3, color="#012f48", 
                           interpolate=True, label='Decrease')
        
        # 绘制曲线
        ax.plot(lat_values_smooth, lat_smooth, color='black', linewidth=1.5)
        
        # 添加零线
        ax.axvline(x=0, color='gray', linestyle='-', alpha=0.5, linewidth=1)
        
    else:
        # 正值数据（如原始投资额）
        x_min = 0
        x_max = max(lat_values_smooth) * 1.1
        
        # 绘制曲线和填充
        ax.plot(lat_values_smooth, lat_smooth, color='#7a0101', linewidth=1.5)
        ax.fill_betweenx(lat_smooth, 0, lat_values_smooth, alpha=0.3, color='#a37070')
    
    # 设置范围
    ax.set_ylim(-90, 90)
    ax.set_xlim(x_min, x_max)
    
    # 设置标题和标签
    ax.set_title(f'Latitude Distribution {title_suffix}', fontsize=12, fontweight='bold', rotation=90)
    # ax.set_ylabel('Latitude (°)', fontsize=10)
    ax.set_ylabel('')  # 移除x轴标题
    ax.set_yticklabels([]) 
    ax.tick_params(axis='y', length=0)
    # 美化图表
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.grid(False)
    
    # 如果有图例，显示它
    # if column in ['diff_absolute', 'diff_relative'] and (np.any(lat_values_smooth > 0) and np.any(lat_values_smooth < 0)):
    #     ax.legend(fontsize=9)
    
    return ax

In [ ]:
# 创建经纬度分布图
print("=" * 60)
print("CREATING PROFILE PLOTS")
print("=" * 60)

print("\n1. Creating baseline investment profiles (2020)...")
create_longitude_profile(df, 'total_cost_2020_ave', ax=None, title_suffix="- 2020 Baseline")
create_latitude_profile(df, 'total_cost_2020_ave', ax=None, title_suffix="- 2020 Baseline")

print("\n2. Creating absolute difference profiles...")
create_longitude_profile(df, 'diff_absolute', ax=None, title_suffix="- Absolute Difference")
create_latitude_profile(df, 'diff_absolute', ax=None, title_suffix="- Absolute Difference")

print("\n3. Creating relative difference profiles...")
create_longitude_profile(df, 'diff_relative', ax=None, title_suffix="- Relative Change")
create_latitude_profile(df, 'diff_relative', ax=None, title_suffix="- Relative Change")

print("\nProfile plots completed! Each shows the geographic distribution of investment changes.")
print("- Red areas: Increases in investment")  
print("- Blue areas: Decreases in investment")
print("- Black line: Overall trend")
print("- Gray line: Zero reference line (for difference plots)")

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import matplotlib.patches as mpatches
from collections import defaultdict
import os

plt.rcParams['font.family'] = 'Times New Roman'

# 1. 读取岛屿数据
filtered_islands = pd.read_csv('../demand_get/filtered_island_1898.csv')
print(f"读取到 {len(filtered_islands)} 个岛屿")

# 2. 读取IPCC区域数据
ipcc_regions = gpd.read_file('../visualization/IPCC-WGI-reference-regions-v4.geojson')
print(f"读取到 {len(ipcc_regions)} 个IPCC区域")

# 3. 将岛屿坐标映射到IPCC区域
def map_islands_to_ipcc_regions(islands_df, ipcc_regions):
    """将岛屿坐标映射到IPCC区域"""
    island_region_mapping = []
    
    for idx, island in islands_df.iterrows():
        lat, lon = island['Lat'], island['Long']
        point = Point(lon, lat)  # 注意经纬度的顺序
        
        # 查找包含该点的IPCC区域
        region_found = False
        for _, region in ipcc_regions.iterrows():
            if region.geometry.contains(point):
                island_region_mapping.append({
                    'island_lat': lat,
                    'island_lon': lon,
                    'region_name': region['Name'],
                    'region_acronym': region['Acronym'],
                    'continent': region['Continent']
                })
                region_found = True
                break
        
        if not region_found:
            island_region_mapping.append({
                'island_lat': lat,
                'island_lon': lon,
                'region_name': 'Unknown',
                'region_acronym': 'UNK',
                'continent': 'Unknown'
            })
    
    return pd.DataFrame(island_region_mapping)

print("正在将岛屿映射到IPCC区域...")
island_region_map = map_islands_to_ipcc_regions(filtered_islands, ipcc_regions)

# 4. 计算每个区域的岛屿数量
region_counts = island_region_map['region_acronym'].value_counts()
print("\n各区域岛屿数量：")
print(region_counts.head(15))

# 5. 筛选至少有10个岛屿的区域
valid_regions = region_counts[region_counts >= 10].index.tolist()
print(f"\n至少有10个岛屿的区域：{valid_regions}")

# 过滤数据只保留有效区域的岛屿
valid_islands = island_region_map[island_region_map['region_acronym'].isin(valid_regions)].copy()
print(f"有效岛屿数量：{len(valid_islands)}")

# 6. 读取每个岛屿的容量数据并计算区域平均值 - 为2020和2050两个年份
def calculate_regional_capacity(valid_islands, year):
    """计算每个有效区域的平均容量，包括所有组件类型"""
    regional_capacity = defaultdict(lambda: {
        'WT': [], 'PV': [], 'WEC': [],  # 可再生能源组件
        'ESS': [], 'TES': [], 'CES': [], 'H2S': [],  # 储能组件
        'LNG': [], 'LNGV': [], 'CHP': [], 'EB': [], 'AC': [], 'PEM': [], 'FC': []  # 其他组件
    })
    
    data_path = '../result'
    capacity_file = f'island_capacity_{year}.csv'
    capacity_file_path = os.path.join(data_path, capacity_file)
    
    # 定义所有组件类型
    all_components = ['WT', 'PV', 'WEC', 'ESS', 'TES', 'CES', 'H2S', 'LNG', 'LNGV', 'CHP', 'EB', 'AC', 'PEM', 'FC']
    
    if os.path.exists(capacity_file_path):
        try:
            # 读取容量数据
            capacity_data = pd.read_csv(capacity_file_path)
            print(f"✓ 读取{year}年容量数据: {len(capacity_data)} 条记录")
            
            # 假设capacity_data包含lat, lon列用于匹配岛屿
            if 'lat' in capacity_data.columns and 'lon' in capacity_data.columns:
                # 将容量数据与岛屿-区域映射合并
                for _, island in valid_islands.iterrows():
                    lat, lon = island['island_lat'], island['island_lon']
                    region_acronym = island['region_acronym']
                    
                    # 查找匹配的容量数据（允许一定误差）
                    tolerance = 0.01
                    matches = capacity_data[
                        (abs(capacity_data['lat'] - lat) < tolerance) & 
                        (abs(capacity_data['lon'] - lon) < tolerance)
                    ]
                    
                    if len(matches) > 0:
                        match = matches.iloc[0]
                        # 添加各组件容量数据
                        for component in all_components:
                            if component in match:
                                regional_capacity[region_acronym][component].append(match[component])
                            else:
                                regional_capacity[region_acronym][component].append(0)
            else:
                print(f"⚠️ {year}年容量数据中没有lat, lon列，使用随机分布")
                regional_capacity = create_mock_capacity_data(valid_islands, all_components)
        except Exception as e:
            print(f"读取{year}年容量文件时出错: {e}")
            regional_capacity = create_mock_capacity_data(valid_islands, all_components)
    else:
        print(f"❌ {year}年容量文件不存在，创建模拟数据")
        regional_capacity = create_mock_capacity_data(valid_islands, all_components)
    
    # 计算每个区域的平均值
    region_avg_capacity = []
    for region, capacity_data in regional_capacity.items():
        if capacity_data['WT']:  # 如果有数据
            avg_capacity = {}
            for component in all_components:
                avg_capacity[f'avg_{component.lower()}_capacity'] = np.mean(capacity_data[component])
            
            region_capacity_data = {'region_acronym': region, 'island_count': len(capacity_data['WT'])}
            region_capacity_data.update(avg_capacity)
            region_avg_capacity.append(region_capacity_data)
    
    return pd.DataFrame(region_avg_capacity)

def create_mock_capacity_data(valid_islands, all_components):
    """创建模拟容量数据"""
    regional_capacity = defaultdict(lambda: {comp: [] for comp in all_components})
    
    for _, island in valid_islands.iterrows():
        region_acronym = island['region_acronym']
        
        # 为每个岛屿生成随机容量
        for component in all_components:
            if component in ['WT', 'PV', 'WEC']:  # 可再生能源
                cap = np.random.exponential(80)
            elif component in ['ESS', 'TES', 'CES', 'H2S']:  # 储能
                cap = np.random.exponential(40)
            else:  # 其他组件
                cap = np.random.exponential(30)
            
            regional_capacity[region_acronym][component].append(cap)
    
    return regional_capacity

# 7. 为每个IPCC区域定义可视化坐标
def get_predefined_region_coordinates():
    """为每个IPCC区域定义固定的可视化坐标位置"""
    region_coords = {
        'CAR': (20.0, -60.0),   # Caribbean
        'NAO': (40.0, -30.0),   # North Atlantic Ocean
        'MED': (35.0, 20.0),    # Mediterranean
        'SSA': (-50.0, -70.0),  # Southern South America
        'NZ': (-40.0, 175.0),   # New Zealand
        'EAS': (30.0, 120.0),   # East Asia
        'SPO': (-40.0, -150.0), # South Pacific Ocean
        'SEA': (0.0, 125.0),    # Southeast Asia
        'NEU': (55.0, 10.0),    # Northern Europe
        'GIC': (65.0, -40.0),   # Greenland/Iceland
        'AUS': (-25.0, 135.0),  # Australia
        'SAO': (-20.0, -10.0),  # South Atlantic Ocean
        'SAS': (20.0, 80.0),    # South Asia
        'WAS': (15.0, 45.0),    # West Asia
        'CAS': (45.0, 70.0),    # Central Asia
        'ENA': (45.0, -75.0),   # Eastern North America
        'WNA': (45.0, -120.0),  # Western North America
        'CNA': (35.0, -100.0),  # Central North America
        'NEN': (65.0, 30.0),    # Northern Europe North
        'WSA': (-10.0, -60.0),  # Western South America
        'NSA': (-5.0, -55.0),   # Northern South America
        'NES': (-15.0, -45.0),  # Northeast South America
        'SAM': (-30.0, -60.0),  # South America
        'WAF': (10.0, 0.0),     # West Africa
        'CAF': (0.0, 20.0),     # Central Africa
        'EAF': (0.0, 40.0),     # East Africa
        'SAF': (-30.0, 25.0),   # Southern Africa
        'MDG': (-20.0, 47.0),   # Madagascar
        'ESB': (70.0, 120.0),   # East Siberia
        'WSB': (65.0, 80.0),    # West Siberia
        'RFE': (55.0, 135.0),   # Russian Far East
        'RAR': (75.0, 105.0),   # Russian Arctic
        'WCA': (40.0, -10.0),   # Western Central Africa
        'ECA': (50.0, 20.0),    # Eastern Central Africa
        'TIB': (32.0, 90.0),    # Tibet
        'EEU': (55.0, 40.0),    # Eastern Europe
        'SWS': (-40.0, -73.0),  # Southwest Scandinavia (修正为南美洲西南部)
        'NWS': (70.0, 15.0),    # Northwest Scandinavia
        'CEU': (50.0, 15.0),    # Central Europe
        'WCE': (45.0, 5.0),     # West Central Europe
        'ECE': (50.0, 25.0),    # East Central Europe
        'MES': (30.0, 50.0),    # Middle East South
        'MEN': (35.0, 35.0),    # Middle East North
        'ARO': (75.0, 0.0),     # Arctic Ocean
        'BOB': (15.0, 90.0),    # Bay of Bengal
        'ARS': (15.0, 50.0),    # Arabian Sea
        'SCS': (15.0, 115.0),   # South China Sea
        'IOD': (-15.0, 75.0),   # Indian Ocean Dipole
        'WIO': (-15.0, 60.0),   # Western Indian Ocean
        'EIO': (-15.0, 90.0),   # Eastern Indian Ocean
        'SIO': (-35.0, 75.0),   # Southern Indian Ocean
        'EPO': (-10.0, -120.0), # Eastern Pacific Ocean
        'NPO': (30.0, -150.0),  # North Pacific Ocean
        'ARP': (15.0, 50.0),    # Arabian Peninsula
        'SAH': (25.0, 10.0),    # Sahara
        'SCA': (10.0, -85.0),   # Southern Central America
        'NWN': (60.0, -140.0),  # Northwest North America
        'NAU': (-15.0, 135.0),  # Northern Australia
    }
    
    return region_coords

def apply_predefined_coordinates(regional_capacity_data):
    """为有效区域应用预定义的可视化坐标"""
    predefined_coords = get_predefined_region_coordinates()
    
    region_coords_list = []
    for _, region_data in regional_capacity_data.iterrows():
        region_acronym = region_data['region_acronym']
        
        if region_acronym in predefined_coords:
            lat, lon = predefined_coords[region_acronym]
            region_coords_list.append({
                'region_acronym': region_acronym,
                'center_lat': lat,
                'center_lon': lon
            })
        else:
            # 如果没有预定义坐标，使用默认位置
            print(f"警告: 区域 {region_acronym} 没有预定义坐标，使用默认位置")
            region_coords_list.append({
                'region_acronym': region_acronym,
                'center_lat': 0.0,
                'center_lon': 0.0
            })
    
    return pd.DataFrame(region_coords_list)

def create_component_pie_chart(final_capacity_data, year, component_category, components, colors, title):
    """基于IPCC区域数据创建指定组件类别的容量饼状图可视化"""
    
    # 构建列名
    component_columns = [f'avg_{comp.lower()}_capacity' for comp in components]
    
    # 计算总容量并按总容量排序，确保小饼图先画，不会被大饼图覆盖
    final_capacity_data['total_capacity'] = sum([final_capacity_data[col] for col in component_columns])
    df_sorted = final_capacity_data.sort_values(by='total_capacity', ascending=True)
    
    # 创建带有 cartopy 投影的图形
    fig = plt.figure(figsize=(16, 10), dpi=300)
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson())
    
    # 设置背景和地图特征
    ax.set_global()
    ax.add_feature(cfeature.LAND, facecolor='#E0E0E0', zorder=0)
    ax.add_feature(cfeature.OCEAN, facecolor='#FFFFFF', zorder=0)
    ax.add_feature(cfeature.COASTLINE, linewidth=0.7, zorder=1)
    ax.add_feature(cfeature.BORDERS, linestyle=':', linewidth=0.5, alpha=0.6, zorder=1)
    
    # 动态调整大小的参数
    base_zoom = 0.12  # 饼图的基础缩放比例
    scale_factor = 0.00005  # 容量对大小的影响因子
    
    # 遍历每个区域来绘制饼图
    for index, row in df_sorted.iterrows():
        lat, lon = row['center_lat'], row['center_lon']
        total_capacity = row['total_capacity']
        region_name = row['region_acronym']
        island_count = row['island_count']
        
        # 如果总容量为0，则跳过
        if total_capacity <= 0:
            continue
        
        # 获取各组件的容量
        capacities = [row[col] for col in component_columns]
        
        # 过滤掉0值，避免空饼块
        non_zero_capacities = []
        non_zero_colors = []
        
        for i, cap in enumerate(capacities):
            if cap > 0:
                non_zero_capacities.append(cap)
                non_zero_colors.append(colors[i])
        
        if not non_zero_capacities:  # 如果没有非零容量
            continue
            
        # 创建一个临时的、透明的画布来生成饼图图像
        fig_temp, ax_temp = plt.subplots(figsize=(2, 2), dpi=150)
        fig_temp.patch.set_alpha(0)  # 图窗背景透明
        ax_temp.patch.set_alpha(0)   # 坐标轴背景透明
        
        # 在临时坐标轴上绘制饼图
        wedges, texts = ax_temp.pie(
            non_zero_capacities,
            colors=non_zero_colors,
            wedgeprops={'edgecolor': 'white', 'linewidth': 1.5},
            startangle=90
        )
        ax_temp.set_aspect('equal')  # 保证是圆形
        ax_temp.axis('off')  # 关闭坐标轴显示
        fig_temp.tight_layout(pad=0)  # 去除白边
        
        # 将绘制好的饼图渲染成一个Numpy数组图像
        fig_temp.canvas.draw()
        pie_img = np.array(fig_temp.canvas.renderer.buffer_rgba())
        plt.close(fig_temp)  # 关闭临时图像，释放内存
        
        # 计算饼图的动态大小 (缩放比例)
        zoom = base_zoom + scale_factor * total_capacity
        zoom = max(0.05, min(zoom, 0.25))  # 限制缩放范围
        
        # 将饼图图像作为OffsetImage添加到主地图上
        # 将地理坐标转换为地图投影坐标
        point_in_map_proj = ax.projection.transform_point(lon, lat, ccrs.Geodetic())
        
        # 创建图像盒子
        imagebox = OffsetImage(pie_img, zoom=zoom)
        imagebox.image.axes = ax
        
        # 使用AnnotationBbox将图像盒子精确放置在地图上
        ab = AnnotationBbox(
            imagebox,
            point_in_map_proj,
            frameon=False,
            pad=0
        )
        ax.add_artist(ab)
        
        # 在饼图旁边标注区域名称
        label_text = f"{region_name}"
        
        # 根据位置调整标签位置
        offset_lon = 5 if lon < 0 else -5
        offset_lat = -5 if lat > 0 else 5
        
        ax.text(lon + offset_lon,
                lat + offset_lat,
                label_text,
                transform=ccrs.Geodetic(),
                ha='center',
                va='center',
                zorder=11,
                fontsize=12,
                bbox=dict(boxstyle="round,pad=0.3",
                          facecolor="white", 
                          edgecolor="gray",
                          alpha=0.8))
    
    # 创建图例
    legend_patches = []
    for i, component in enumerate(components):
        legend_patches.append(mpatches.Patch(color=colors[i], label=component))
    
    ax.legend(handles=legend_patches, 
              loc='lower left', 
              bbox_to_anchor=(0.02, 0.02), 
              frameon=True,
              edgecolor='black',
              fontsize=14)
    
    # 添加标题
    # ax.set_title(f'{title} ({year})', 
    #             fontsize=14, pad=15)
    
    # 移除边框
    plt.setp(ax.spines.values(), visible=False)
    
    plt.tight_layout()
    plt.show()
    
    return fig

# 定义三种组件类别及其颜色
component_categories = {
    'renewable': {
        'components': ['WT', 'PV', 'WEC'],
        'colors': ["lightblue", "#ffcc00", "#17becf"],  # 蓝色，金黄色，青色
        'title': 'Renewable Energy Components'
    },
    'storage': {
        'components': ['ESS', 'TES', 'CES', 'H2S'],
        'colors': ["#1f77b4", "#ff7f0e", "#aec7e8", "#98df8a"],  # 深蓝，橙色，浅蓝，绿色
        'title': 'Energy Storage Components'
    },
    'conversion': {
        'components': ['LNG', 'LNGV', 'CHP', 'EB', 'AC', 'PEM', 'FC'],
        'colors': ["#808080", "#8c564b", "#d62728", "#ff9999", "#7FCDFF", "#2ca02c", "#9467bd"],  # 灰色，棕色，红色，浅红，浅蓝，绿色，紫色
        'title': 'Energy Conversion Components'
    }
}

# 执行容量数据处理和可视化
print("\n正在处理free,2020年和2050年的容量数据...")

for year in [0, 2020, 2050]:
    print(f"\n{'='*60}")
    print(f"处理{year}年数据")
    print(f"{'='*60}")
    
    # 计算区域平均容量
    regional_capacity_data = calculate_regional_capacity(valid_islands, year)
    
    if len(regional_capacity_data) > 0:
        print(f"\n{year}年各区域平均容量数据计算完成")
        
        # 应用预定义坐标
        capacity_coordinates = apply_predefined_coordinates(regional_capacity_data)
        
        # 合并数据用于可视化
        final_capacity_data = regional_capacity_data.merge(capacity_coordinates, on='region_acronym')
        
        # 为三种组件类别分别创建饼图
        for category_name, category_info in component_categories.items():
            print(f"\n正在创建{year}年{category_info['title']}饼状图...")
            
            fig = create_component_pie_chart(
                final_capacity_data.copy(),
                year,
                category_name,
                category_info['components'],
                category_info['colors'],
                category_info['title']
            )
            
            print(f"{year}年{category_info['title']}饼状图完成！")
        
        # 显示统计信息
        print(f"\n=== {year}年容量统计信息 ===")
        print(f"有效IPCC区域数量: {len(final_capacity_data)}")
        print(f"包含的岛屿总数: {final_capacity_data['island_count'].sum()}")
        print(f"平均每区域岛屿数: {final_capacity_data['island_count'].mean():.1f}")
        
        # 计算各组件类别的总容量
        for category_name, category_info in component_categories.items():
            component_columns = [f'avg_{comp.lower()}_capacity' for comp in category_info['components']]
            total_capacities = [final_capacity_data[col].sum() for col in component_columns]
            grand_total = sum(total_capacities)
            
            print(f"\n{category_info['title']}总容量:")
            for i, comp in enumerate(category_info['components']):
                cap = total_capacities[i]
                percentage = cap/grand_total*100 if grand_total > 0 else 0
                print(f"{comp}: {cap:.1f} ({percentage:.1f}%)")
    
    else:
        print(f"没有找到符合条件的{year}年区域容量数据！")

print(f"\n{'='*60}")
print("所有饼状图创建完成！")
print("共创建了6张图：")
print("- 2020年：可再生能源组件 (WT, PV, WEC)")
print("- 2020年：储能组件 (ESS, TES, CES, H2S)")
print("- 2020年：能源转换组件 (LNG, LNGV, CHP, EB, AC, PEM, FC)")
print("- 2050年：可再生能源组件 (WT, PV, WEC)")
print("- 2050年：储能组件 (ESS, TES, CES, H2S)")
print("- 2050年：能源转换组件 (LNG, LNGV, CHP, EB, AC, PEM, FC)")
print(f"{'='*60}")